In [1]:
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten, Dropout

# Load the VGG16 model
vgg = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the layers in the base model
for layer in vgg.layers:
    layer.trainable = False

# Add custom layers on top of VGG16
x = Flatten()(vgg.output)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(3, activation='softmax')(x)  # 3 classes: COVID-19, Pneumonia, Normal

# Create the model
model = Model(inputs=vgg.input, outputs=x)

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [3]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    validation_split=0.2  # 20% of the data for validation
)

train_generator = train_datagen.flow_from_directory(
    r'C:\Users\riyab\Documents\My PROJECTS\DL_PROJECT\Radiography_Dataset',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='training'  # Use the 'training' subset
)

validation_generator = train_datagen.flow_from_directory(
    r'C:\Users\riyab\Documents\My PROJECTS\DL_PROJECT\Radiography_Dataset',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='validation'  # Use the 'validation' subset
)


Found 10350 images belonging to 3 classes.
Found 2586 images belonging to 3 classes.


In [ ]:
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size,
    epochs=20
)

Epoch 1/20


C:\Users\riyab\anaconda3\lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


323/323 ━━━━━━━━━━━━━━━━━━━━ 2146s 7s/step - accuracy: 0.8528 - loss: 0.9489 - val_accuracy: 0.6262 - val_loss: 3.9717
Epoch 2/20
  1/323 ━━━━━━━━━━━━━━━━━━━━ 22:52 4s/step - accuracy: 0.8438 - loss: 0.3803

C:\Users\riyab\anaconda3\lib\contextlib.py:137: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


323/323 ━━━━━━━━━━━━━━━━━━━━ 9s 14ms/step - accuracy: 0.8438 - loss: 0.3803 - val_accuracy: 0.5000 - val_loss: 4.5391
Epoch 3/20
323/323 ━━━━━━━━━━━━━━━━━━━━ 4357s 13s/step - accuracy: 0.9144 - loss: 0.2034 - val_accuracy: 0.5625 - val_loss: 4.5691
Epoch 4/20
323/323 ━━━━━━━━━━━━━━━━━━━━ 8s 11ms/step - accuracy: 0.9375 - loss: 0.2272 - val_accuracy: 0.6154 - val_loss: 4.2863
Epoch 5/20
323/323 ━━━━━━━━━━━━━━━━━━━━ 1954s 6s/step - accuracy: 0.9182 - loss: 0.1812 - val_accuracy: 0.5887 - val_loss: 5.3620
Epoch 6/20
323/323 ━━━━━━━━━━━━━━━━━━━━ 9s 12ms/step - accuracy: 0.8750 - loss: 0.3246 - val_accuracy: 0.7308 - val_loss: 3.4289
Epoch 7/20
323/323 ━━━━━━━━━━━━━━━━━━━━ 1963s 6s/step - accuracy: 0.9209 - loss: 0.1737 - val_accuracy: 0.5938 - val_loss: 5.4899
Epoch 8/20
323/323 ━━━━━━━━━━━━━━━━━━━━ 8s 12ms/step - accuracy: 0.8125 - loss: 0.2360 - val_accuracy: 0.6923 - val_loss: 4.1571
Epoch 9/20
323/323 ━━━━━━━━━━━━━━━━━━━━ 1973s 6s/step - accuracy: 0.9283 - loss: 0.1647 - val_accuracy: 

In [1]:
loss, accuracy = model.evaluate(validation_generator)
print(f'Validation Accuracy: {accuracy * 100:.2f}%')

NameError: name 'model' is not defined

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np

# Predict on validation data
Y_pred = model.predict(validation_generator)
y_pred = np.argmax(Y_pred, axis=1)

print('Confusion Matrix')
print(confusion_matrix(validation_generator.classes, y_pred))

print('Classification Report')
target_names = ['COVID-19', 'Normal', 'Pneumonia']
print(classification_report(validation_generator.classes, y_pred, target_names=target_names))
